In [ ]:
import subprocess, winreg, re, paramiko
from tkinter import *
from tkinter import ttk
import sqlite3
from openpyxl import Workbook

root = Tk()


class Funcs():
    def connect(self):
        self.conn = sqlite3.connect("inventário.bd")
        self.cursor = self.conn.cursor()# ; print('conectado')
    def disconnect(self):
        self.conn.close()# ; print('desconectado')
    def tabelas(self):
        self.connect()
        self.cursor.execute(""" Create table if not exists inventario ( 
            idmaq int(3),
            hostname CHAR(15) PRIMARY KEY,
            ip CHAR(20) NOT NULL,
            user CHAR(35) NOT NULL,
            so CHAR(50) NOT NULL,
            versao CHAR(15) NOT NULL,
            arch CHAR(15) NOT NULL,
            office CHAR(15) NOT NULL,
            ram CHAR(15) NOT NULL,
            disc CHAR(15) NOT NULL,
            model CHAR(35) NOT NULL,
            processor CHAR(50) NOT NULL
                  ); """)
        self.conn.commit()# ; print('Tab criada')
        self.disconnect()
    def select(self):
        try:
            self.connect()
            self.host = [(self.maq)]
            self.cursor.execute(""" select * from inventario where hostname = ? """, self.host)
            self.result = self.cursor.fetchall()
            self.disconnect()
        except:
            pass
        else:
            return self.result


class Prog():
    def dados(self):
        try:
            pd3 = subprocess.run(f'ping -n 1 {self.maq}', stdout=subprocess.PIPE)
            rest3 = pd3.stdout.split()
        except:
            return False
        else:
            self.stdoutd3 = rest3
            if self.stdoutd3[12] == b'TTL=128':
                return self.stdoutd3
            elif self.stdoutd3[12] == b'TTL=64':
                return self.stdoutd3
            elif self.stdoutd3[12] == b'TTL=255':
                return self.stdoutd3
            else:
                return False

    def dados1(self):
        try:
            pd1 = subprocess.run(f'wmic /node: {self.maq} OS get caption', stdout=subprocess.PIPE)
            rest1 = pd1.stdout.decode('UTF-8').split()
            self.stdout1 = ' '.join(rest1[1:])
        except:
            pass
        else:
            return self.stdout1

    def dados2(self):
        try:
            pd2 = subprocess.run(f'wmic /node: {self.maq} OS get Version, TotalVisibleMemorySize, OSArchitecture',
                                 stdout=subprocess.PIPE)
            rest2 = pd2.stdout.decode('UTF-8').split()
        except:
            pass
        else:
            self.stdoutd2 = rest2
            return self.stdoutd2

    def cmd1(self):
        try:
            p1 = subprocess.run(f'wmic /node: {self.maq} computersystem get Caption, Manufacturer, Model, username',
                                stdout=subprocess.PIPE)
            res1 = p1.stdout.decode('UTF-8').split()
        except:
            pass
        else:
            self.stdout1 = res1
            return self.stdout1

    def cmd2(self):
        try:
            p2 = subprocess.run(f'wmic /node: {self.maq} logicaldisk get Name, size', stdout=subprocess.PIPE)
            res2 = p2.stdout.decode('UTF-8').split()
            a = re.match("(\d+)", res2[3])
            b = int(a[0])
        except:
            pass
        else:
            self.stdout2 = (f'{b / 1024**4:.2f} TB') if b > (1024**4) else (f'{b / 1024**3:.2f} GB')
            return self.stdout2

    def winreg_office(self):
        try:
            target = r'\\{}'.format(self.maq)
            off_reg = winreg.ConnectRegistry(target, winreg.HKEY_LOCAL_MACHINE)
            key = winreg.OpenKey(off_reg, r"SOFTWARE\Classes\Word.Application\CurVer")
            value = winreg.QueryValueEx(key, None)[0]
            winreg.CloseKey(key)
        except:
            pass
        else:
            self.office = value[-2:]
            return self.office

    def winreg_processor_brand(self):
        try:
            target = r'\\{}'.format(self.maq)
            rem_reg = winreg.ConnectRegistry(target, winreg.HKEY_LOCAL_MACHINE)
            key = winreg.OpenKey(rem_reg, r"Hardware\Description\System\CentralProcessor\0")
            value = winreg.QueryValueEx(key, "ProcessorNameString")[0]
            winreg.CloseKey(key)
        except:
            pass
        else:
            self.processor_brand = value
            return self.processor_brand.strip()

    def linux(self):
        try:
            self.ssh = paramiko.SSHClient()
            self.ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            self.ssh.connect(f'{self.maq}', username='pi', password='123')
        except:
            pass
        else:
            self.a3 = self.linso()
            self.a4 = self.linver()
            self.a5 = self.linarq()
            self.a7 = self.linmem()
            self.a9 = self.linmod()
            self.a10 = self.linprc()
            self.ssh.close()

    def linso(self):
        stdin, stdout, stderr = self.ssh.exec_command('lsb_release -a | grep Description')
        for line in stdout:
            a = (line.strip(f'\n \l').split())
        return ' '.join(a[1:])

    def linver(self):
        stdin, stdout, stderr = self.ssh.exec_command('uname -r')
        for line in stdout:
            b = (line.strip('\n'))
        return b

    def linarq(self):
        stdin, stdout, stderr = self.ssh.exec_command('getconf LONG_BIT')
        for line in stdout:
            c = (line.strip('\n'))
            a5 = (f'{c} bit')
        return a5

    def linmem(self):
        stdin, stdout, stderr = self.ssh.exec_command('grep MemTotal /proc/meminfo')
        for line in stdout:
            d = (line.strip('\n').split())
            a = re.match("(\d+)", d[-2])
            b = round(int(a[0]))
            a7 = (f'{b / 1024 ** 2:.0f} GB RAM')
        return a7

    def linmod(self):
        stdin, stdout, stderr = self.ssh.exec_command('grep "Model" /proc/cpuinfo')
        for line in stdout:
            e = (line.strip('\n').split())
        return ' '.join(e[2:])

    def linprc(self):
        stdin, stdout, stderr = self.ssh.exec_command('lscpu | grep "Model name"')
        for line in stdout:
            f = (line.strip().split())
        return ' '.join(f[2:])


class App(Prog, Funcs):
    def __init__(self):
        self.root = root
        self.tabelas()
        self.tela()
        self.frames()
        self.labels()
        self.lista()
        root.mainloop()

    def tela(self):
        self.root.title('InvenTI - Inventário de Máquinas')
        self.root.configure(background='#1e3743')
        self.root.geometry("1900x900")

    def frames(self):
        self.frame = Frame(self.root)
        self.frame.place(relx=0.0, rely=0.0, relwidth=1, relheight=1)

    def labels(self):
        self.listac = ttk.Treeview(self.frame, height=15, column=('col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7'
                                                                  , 'col8', 'col9', 'col10', 'col11', ))
        self.h1 = ('id', 'Nome Máquina', 'IP', 'Usuário', 'SO', 'Versão', 'Arquitetura', 'Office', 'Memória', 'Disco',
                   'Modelo', 'Processador')

        self.listac.heading('#0', text='')
        self.listac.heading('#1', text=f'{self.h1[0]}')
        self.listac.heading('#2', text=f'{self.h1[1]}')
        self.listac.heading('#3', text=f'{self.h1[2]}')
        self.listac.heading('#4', text=f'{self.h1[3]}')
        self.listac.heading('#5', text=f'{self.h1[4]}')
        self.listac.heading('#6', text=f'{self.h1[5]}')
        self.listac.heading('#7', text=f'{self.h1[6]}')
        self.listac.heading('#8', text=f'{self.h1[7]}')
        self.listac.heading('#9', text=f'{self.h1[8]}')
        self.listac.heading('#10', text=f'{self.h1[9]}')
        self.listac.heading('#11', text=f'{self.h1[10]}')
        self.listac.heading('#12', text=f'{self.h1[11]}')

        self.listac.column('#0', width=1)
        self.listac.column('#1', width=50)
        self.listac.column('#2', width=100)
        self.listac.column('#3', width=100)
        self.listac.column('#4', width=150)
        self.listac.column('#5', width=200)
        self.listac.column('#6', width=70)
        self.listac.column('#7', width=70)
        self.listac.column('#8', width=100)
        self.listac.column('#9', width=80)
        self.listac.column('#10', width=80)
        self.listac.column('#11', width=250)
        self.listac.column('#12', width=250)
        self.listac.place(relx=0.0, rely=0.048, relwidth=1, relheight=0.952)

        self.scroll = Scrollbar(self.frame, orient='vertical')
        self.listac.configure(yscroll=self.scroll.set)
        self.scroll.place(relx=0.99, rely=0.049, relwidth=0.009, relheight=0.95)

        ry = 0.01
        self.entfax = Entry(self.frame)
        self.entfax.insert(0, '192.168.1')
        self.entfax.place(relx=0.115, rely=ry, relwidth=0.082, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Definir Faixa', command=self.faixa)
        self.bt_frame.place(relx=0.20, rely=ry, relwidth=0.06, relheight=0.025)

        self.entran1 = Entry(self.frame)
        self.entran1.insert(0, '1')
        self.entran1.place(relx=0.330, rely=ry, relwidth=0.032, relheight=0.025)
        self.entran2 = Entry(self.frame)
        self.entran2.insert(0, '255')
        self.entran2.place(relx=0.365, rely=ry, relwidth=0.032, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Definir Range', command=self.range)
        self.bt_frame.place(relx=0.40, rely=ry, relwidth=0.06, relheight=0.025)

        self.entmaq = Entry(self.frame)
        self.entmaq.place(relx=0.515, rely=ry, relwidth=0.082, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Inserir IP', command=self.insm)
        self.bt_frame.place(relx=0.60, rely=ry, relwidth=0.06, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Apagar IP', command=self.delm)
        self.bt_frame.place(relx=0.663, rely=ry, relwidth=0.06, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Atualizar tudo', command=self.atup)
        self.bt_frame.place(relx=0.8, rely=ry, relwidth=0.06, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Extrair Excel', command=self.openxl)
        self.bt_frame.place(relx=0.863, rely=ry, relwidth=0.06, relheight=0.025)

    def lista(self):
        self.listac.delete(*self.listac.get_children())
        self.connect()
        self.listasel = self.cursor.execute(""" select * from inventario order by idmaq, ip """)
        for m in self.listasel:
            self.listac.insert('', END, values=m)
        self.disconnect()

    def listaex(self):
        self.connect()
        self.listasel = self.cursor.execute(""" select * from inventario order by idmaq, ip """)
        for linha in self.listasel:
            self.plan1.append(linha)
        self.disconnect()

    def atup(self):
        self.maq = ''
        self.faixa()
        self.range()
        for x in range(self.ran1, self.ran2):
            self.maq = f'{self.fax}.{x}'
            print(self.maq)
            self.verificarSistema()
        self.lista()

    def insm(self):
        self.maq = ''
        self.maq = self.entmaq.get()
        self.verificarSistema()
        self.lista()

    def delm(self):
        self.maq = ''
        self.maq = (self.entmaq.get().split())
        self.connect()
        self.sqli = """ delete from inventario where ip = ?; """
        self.cursor.execute(self.sqli, self.maq)
        self.conn.commit()# ; print(f'apagado maq {self.maq}')
        self.disconnect()
        self.lista()

    def faixa(self):
        self.fax = ''
        self.fax = self.entfax.get()

    def range(self):
        self.ran1 = 1
        self.ran1 = int(self.entran1.get())
        self.ran2 = 255
        self.ran2 = int(self.entran2.get())+1

    def openxl(self):
        self.arqexc = Workbook()
        self.plan1 = self.arqexc.active
        self.plan1.title = 'Inventário'
        self.plan1['A1'] = self.h1[0]
        self.plan1['B1'] = self.h1[1]
        self.plan1['C1'] = self.h1[2]
        self.plan1['D1'] = self.h1[3]
        self.plan1['E1'] = self.h1[4]
        self.plan1['F1'] = self.h1[5]
        self.plan1['G1'] = self.h1[6]
        self.plan1['H1'] = self.h1[7]
        self.plan1['I1'] = self.h1[8]
        self.plan1['J1'] = self.h1[9]
        self.plan1['K1'] = self.h1[10]
        self.plan1['L1'] = self.h1[11]

        self.listaex()
        self.arqexc.save(r"c:\InvenTI\relatorio.xlsx")

    def verificarSistema(self):
        self.reg = '0'
        self.ip = self.dados()
        self.id = (self.ip[1][10:])
        if self.ip == False:
            self.status = 'Offline'

        elif self.ip[12] == b'TTL=64':
            self.params = self.linux()
            self.a1 = self.ip[1].decode('utf-8')
            self.a0 = 'MAQ' + self.ip[1][10:].decode('utf-8') + ' Linux'
            self.a2 = self.a6 = self.a8 = 'Sem Ref.'
            self.status = "Online"
            self.parametros()

        elif self.ip[12] == b'TTL=255':
            self.a1 = self.ip[1].decode('utf-8')
            self.a0 = 'MAQ' + self.ip[1][10:].decode('utf-8') + ' Imp'
            self.a2 = self.a3 = self.a4 = self.a5 = self.a6 = self.a7 = self.a8 = self.a9 = self.a10 = 'Sem Ref.'
            self.status = "Online"
            self.parametros()

        elif self.ip[12] == b'TTL=128':
            self.reg = self.winreg_processor_brand()
            if self.reg == "0" or self.reg == None:
                self.a1 = self.ip[1].decode('utf-8')
                self.a0 = 'MAQ' + self.ip[1][10:].decode('utf-8') + ' Indef'
                self.a2 = self.a3 = self.a4 = self.a5 = self.a6 = self.a7 = self.a8 = self.a9 = self.a10 = 'Sem Ref.'
                self.status = "Online"
                self.parametros()
            else:
                self.os = self.dados1()
                self.ver = self.dados2()
                a = re.match("(\d+)", self.ver[-2])
                b = round(int(a[0]))
                self.mem = (f'{b / 1024 ** 2:.0f}')
                self.user = self.cmd1()
                self.disc = self.cmd2()
                self.off = self.winreg_office()

                self.a0 = str(self.user[4])
                self.a1 = self.ip[1].decode('utf-8')
                if 'CEDASA' in self.user[-1]:
                    self.a2 = self.user[-1]
                    self.a9 = ' '.join(self.user[6:-1]) if self.user[5] == self.user[6] else ' '.join(self.user[5:-1])
                else:
                    self.a2 = 'CEDASA\Administrator'
                    self.a9 = ' '.join(self.user[6:]) if self.user[5] == self.user[6] else ' '.join(self.user[5:])
                self.a3 = self.os
                self.a4 = self.ver[-1]
                self.a5 = ' '.join(self.ver[3:-2])
                if self.off == "10":
                    self.a6 = 'Office XP'
                elif self.off == "11":
                    self.a6 = 'Office 2003'
                elif self.off == "12":
                    self.a6 = 'Office 2007'
                elif self.off == "14":
                    self.a6 = 'Office 2010'
                elif self.off == "15":
                    self.a6 = 'Office 2013'
                elif self.off == "16":
                    self.a6 = 'Office 365/2016'
                else:
                    self.a6 = 'Não reconhecido'
                self.a7 = self.mem + ' GB RAM'
                if self.disc == None:
                    self.a8 = 'Virtual Machine'
                else:
                    self.a8 = self.disc
                self.a10 = self.reg
                self.status = "Online"
                self.parametros()


    def parametros(self):
        self.tpl = (self.id, self.a0, self.a1, self.a2, self.a3, self.a4, self.a5, self.a6, self.a7, self.a8, self.a9,
                    self.a10)
        # print(self.tpl)
        self.insert()

    def insert(self):
        self.connect()
        self.sqli = """ insert or replace into inventario (idmaq, hostname, ip, user, so, versao, arch, office, ram, 
                        disc, model, processor) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?); """
        self.cursor.execute(self.sqli, self.tpl)
        self.conn.commit() # ; print(f'inserido maq {self.maq}')
        self.disconnect()


App()

In [ ]:
import subprocess, winreg, re, paramiko
from tkinter import *
from tkinter import ttk
import sqlite3
from openpyxl import Workbook

root = Tk()


class Funcs():
    def connect(self):
        self.conn = sqlite3.connect("../Scripts/inventário.bd")
        self.cursor = self.conn.cursor(); print('conectado')
    def disconnect(self):
        self.conn.close(); print('desconectado')
    def tabelas(self):
        self.connect()
        self.cursor.execute(""" Create table if not exists inventario ( 
            hostname CHAR(15) PRIMARY KEY,
            ip CHAR(20) NOT NULL,
            user CHAR(35) NOT NULL,
            so CHAR(50) NOT NULL,
            versao CHAR(15) NOT NULL,
            arch CHAR(15) NOT NULL,
            office CHAR(15) NOT NULL,
            ram CHAR(15) NOT NULL,
            disc CHAR(15) NOT NULL,
            model CHAR(35) NOT NULL,
            processor CHAR(50) NOT NULL
                  ); """)
        self.conn.commit(); print('Tab criada')
        self.disconnect()
    def select(self):
        try:
            self.connect()
            self.host = [(self.maq)]
            self.cursor.execute(""" select * from inventario where hostname = ? """, self.host)
            self.result = self.cursor.fetchall()
            self.disconnect()
        except:
            pass
        else:
            return self.result


class Prog():
    def dados(self):
        try:
            pd3 = subprocess.run(f'ping -n 1 {self.maq}', stdout=subprocess.PIPE)
            rest3 = pd3.stdout.split()
        except:
            return False
        else:
            self.stdoutd3 = rest3
            if self.stdoutd3[12] == b'TTL=128':
                return self.stdoutd3
            elif self.stdoutd3[12] == b'TTL=64':
                return self.stdoutd3
            elif self.stdoutd3[12] == b'TTL=255':
                return self.stdoutd3
            else:
                return False

    def dados1(self):
        try:
            pd1 = subprocess.run(f'wmic /node: {self.maq} OS get caption', stdout=subprocess.PIPE)
            rest1 = pd1.stdout.decode('UTF-8').split()
            self.stdout1 = ' '.join(rest1[1:])
        except:
            pass
        else:
            return self.stdout1

    def dados2(self):
        try:
            pd2 = subprocess.run(f'wmic /node: {self.maq} OS get Version, TotalVisibleMemorySize, OSArchitecture',
                                 stdout=subprocess.PIPE)
            rest2 = pd2.stdout.decode('UTF-8').split()
        except:
            pass
        else:
            self.stdoutd2 = rest2
            return self.stdoutd2

    def cmd1(self):
        try:
            p1 = subprocess.run(f'wmic /node: {self.maq} computersystem get Caption, Manufacturer, Model, username',
                                stdout=subprocess.PIPE)
            res1 = p1.stdout.decode('UTF-8').split()
        except:
            pass
        else:
            self.stdout1 = res1
            return self.stdout1

    def cmd2(self):
        try:
            p2 = subprocess.run(f'wmic /node: {self.maq} logicaldisk get Name, size', stdout=subprocess.PIPE)
            res2 = p2.stdout.decode('UTF-8').split()
            a = re.match("(\d+)", res2[3])
            b = int(a[0])
        except:
            pass
        else:
            self.stdout2 = (f'{b / 1024**4:.2f} TB') if b > (1024**4) else (f'{b / 1024**3:.2f} GB')
            return self.stdout2

    def winreg_office(self):
        try:
            target = r'\\{}'.format(self.maq)
            off_reg = winreg.ConnectRegistry(target, winreg.HKEY_LOCAL_MACHINE)
            key = winreg.OpenKey(off_reg, r"SOFTWARE\Classes\Word.Application\CurVer")
            value = winreg.QueryValueEx(key, None)[0]
            winreg.CloseKey(key)
        except:
            pass
        else:
            self.office = value[-2:]
            return self.office

    def winreg_processor_brand(self):
        try:
            target = r'\\{}'.format(self.maq)
            rem_reg = winreg.ConnectRegistry(target, winreg.HKEY_LOCAL_MACHINE)
            key = winreg.OpenKey(rem_reg, r"Hardware\Description\System\CentralProcessor\0")
            value = winreg.QueryValueEx(key, "ProcessorNameString")[0]
            winreg.CloseKey(key)
        except:
            pass
        else:
            self.processor_brand = value
            return self.processor_brand.strip()

    def linux(self):
        try:
            self.ssh = paramiko.SSHClient()
            self.ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            self.ssh.connect(f'{self.maq}', username='pi', password='123')
        except:
            pass
        else:
            self.a3 = self.linso()
            self.a4 = self.linver()
            self.a5 = self.linarq()
            self.a7 = self.linmem()
            self.a9 = self.linmod()
            self.a10 = self.linprc()
            self.ssh.close()

    def linso(self):
        stdin, stdout, stderr = self.ssh.exec_command('lsb_release -a | grep Description')
        for line in stdout:
            a = (line.strip(f'\n \l').split())
        return ' '.join(a[1:])

    def linver(self):
        stdin, stdout, stderr = self.ssh.exec_command('uname -r')
        for line in stdout:
            b = (line.strip('\n'))
        return b

    def linarq(self):
        stdin, stdout, stderr = self.ssh.exec_command('getconf LONG_BIT')
        for line in stdout:
            c = (line.strip('\n'))
            a5 = (f'{c} bit')
        return a5

    def linmem(self):
        stdin, stdout, stderr = self.ssh.exec_command('grep MemTotal /proc/meminfo')
        for line in stdout:
            d = (line.strip('\n').split())
            a = re.match("(\d+)", d[-2])
            b = round(int(a[0]))
            a7 = (f'{b / 1024 ** 2:.0f} GB RAM')
        return a7

    def linmod(self):
        stdin, stdout, stderr = self.ssh.exec_command('grep "Model" /proc/cpuinfo')
        for line in stdout:
            e = (line.strip('\n').split())
        return ' '.join(e[2:])

    def linprc(self):
        stdin, stdout, stderr = self.ssh.exec_command('lscpu | grep "Model name"')
        for line in stdout:
            f = (line.strip().split())
        return ' '.join(f[2:])


class App(Prog, Funcs):
    def __init__(self):
        self.root = root
        self.tabelas()
        self.tela()
        self.frames()
        self.labels()
        self.lista()
        root.mainloop()

    def tela(self):
        self.root.title('Inventário de Máquinas')
        self.root.configure(background='#1e3743')
        self.root.geometry("1900x900")

    def frames(self):
        self.frame = Frame(self.root)
        self.frame.place(relx=0.0, rely=0.0, relwidth=1, relheight=1)

    def labels(self):
        self.listac = ttk.Treeview(self.frame, height=15, column=('col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7'
                                                                  , 'col8', 'col9', 'col10', 'col11'))
        self.h1 = ('Nome Máquina', 'IP', 'Usuário', 'SO', 'Versão', 'Arquitetura', 'Office', 'Memória', 'Disco',
                   'Modelo', 'Processador')

        self.listac.heading('#0', text='')
        self.listac.heading('#1', text=f'{self.h1[0]}')
        self.listac.heading('#2', text=f'{self.h1[1]}')
        self.listac.heading('#3', text=f'{self.h1[2]}')
        self.listac.heading('#4', text=f'{self.h1[3]}')
        self.listac.heading('#5', text=f'{self.h1[4]}')
        self.listac.heading('#6', text=f'{self.h1[5]}')
        self.listac.heading('#7', text=f'{self.h1[6]}')
        self.listac.heading('#8', text=f'{self.h1[7]}')
        self.listac.heading('#9', text=f'{self.h1[8]}')
        self.listac.heading('#10', text=f'{self.h1[9]}')
        self.listac.heading('#11', text=f'{self.h1[10]}')

        self.listac.column('#0', width=1)
        self.listac.column('#1', width=100)
        self.listac.column('#2', width=100)
        self.listac.column('#3', width=150)
        self.listac.column('#4', width=200)
        self.listac.column('#5', width=70)
        self.listac.column('#6', width=70)
        self.listac.column('#7', width=100)
        self.listac.column('#8', width=80)
        self.listac.column('#9', width=80)
        self.listac.column('#10', width=250)
        self.listac.column('#11', width=250)
        self.listac.place(relx=0.0, rely=0.048, relwidth=1, relheight=0.952)

        self.scroll = Scrollbar(self.frame, orient='vertical')
        self.listac.configure(yscroll=self.scroll.set)
        self.scroll.place(relx=0.99, rely=0.049, relwidth=0.009, relheight=0.95)

        ry = 0.01
        self.entfax = Entry(self.frame)
        self.entfax.insert(0, '192.168.1')
        self.entfax.place(relx=0.115, rely=ry, relwidth=0.082, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Definir Faixa', command=self.faixa)
        self.bt_frame.place(relx=0.20, rely=ry, relwidth=0.06, relheight=0.025)

        self.entran1 = Entry(self.frame)
        self.entran1.insert(0, '1')
        self.entran1.place(relx=0.330, rely=ry, relwidth=0.032, relheight=0.025)
        self.entran2 = Entry(self.frame)
        self.entran2.insert(0, '255')
        self.entran2.place(relx=0.365, rely=ry, relwidth=0.032, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Definir Range', command=self.range)
        self.bt_frame.place(relx=0.40, rely=ry, relwidth=0.06, relheight=0.025)

        self.entmaq = Entry(self.frame)
        self.entmaq.place(relx=0.515, rely=ry, relwidth=0.082, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Inserir IP', command=self.insm)
        self.bt_frame.place(relx=0.60, rely=ry, relwidth=0.06, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Apagar IP', command=self.delm)
        self.bt_frame.place(relx=0.663, rely=ry, relwidth=0.06, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Atualizar tudo', command=self.atup)
        self.bt_frame.place(relx=0.8, rely=ry, relwidth=0.06, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Extrair Excel', command=self.openxl)
        self.bt_frame.place(relx=0.863, rely=ry, relwidth=0.06, relheight=0.025)

    def lista(self):
        self.listac.delete(*self.listac.get_children())
        self.connect()
        self.listasel = self.cursor.execute(""" select * from inventario order by hostname """)
        for m in self.listasel:
            self.listac.insert('', END, values=m)
        self.disconnect()

    def listaex(self):
        self.connect()
        self.listasel = self.cursor.execute(""" select * from inventario order by hostname """)
        for linha in self.listasel:
            self.plan1.append(linha)
        self.disconnect()

    def atup(self):
        self.maq = ''
        self.faixa()
        self.range()
        for x in range(self.ran1, self.ran2):
            self.maq = f'{self.fax}.{x}'
            print(self.maq)
            self.verificarSistema()
        self.lista()

    def insm(self):
        self.maq = ''
        self.maq = self.entmaq.get()
        self.verificarSistema()
        self.lista()

    def delm(self):
        self.maq = ''
        self.maq = (self.entmaq.get().split())
        self.connect()
        self.sqli = """ delete from inventario where ip = ?; """
        self.cursor.execute(self.sqli, self.maq)
        self.conn.commit(); print(f'apagado maq {self.maq}')
        self.disconnect()
        self.lista()

    def faixa(self):
        self.fax = ''
        self.fax = self.entfax.get()

    def range(self):
        self.ran1 = 1
        self.ran1 = int(self.entran1.get())
        self.ran2 = 255
        self.ran2 = int(self.entran2.get())+1

    def openxl(self):
        self.arqexc = Workbook()
        self.plan1 = self.arqexc.active
        self.plan1.title = 'Inventário'
        self.plan1['A1'] = self.h1[0]
        self.plan1['B1'] = self.h1[1]
        self.plan1['C1'] = self.h1[2]
        self.plan1['D1'] = self.h1[3]
        self.plan1['E1'] = self.h1[4]
        self.plan1['F1'] = self.h1[5]
        self.plan1['G1'] = self.h1[6]
        self.plan1['H1'] = self.h1[7]
        self.plan1['I1'] = self.h1[8]
        self.plan1['J1'] = self.h1[9]
        self.plan1['K1'] = self.h1[10]

        self.listaex()
        self.arqexc.save(r"c:\temp\relatorio.xlsx")

    def verificarSistema(self):
        self.reg = '0'
        self.ip = self.dados()
        if self.ip == False:
            self.status = 'Offline'

        elif self.ip[12] == b'TTL=64':
            self.params = self.linux()
            self.a1 = self.ip[1].decode('utf-8')
            self.a0 = 'MAQ' + self.ip[1][10:].decode('utf-8') + ' Linux'
            self.a2 = self.a6 = self.a8 = 'Sem Ref.'
            self.status = "Online"
            self.parametros()

        elif self.ip[12] == b'TTL=255':
            self.a1 = self.ip[1].decode('utf-8')
            self.a0 = 'MAQ' + self.ip[1][10:].decode('utf-8') + ' Imp'
            self.a2 = self.a3 = self.a4 = self.a5 = self.a6 = self.a7 = self.a8 = self.a9 = self.a10 = 'Sem Ref.'
            self.status = "Online"
            self.parametros()

        elif self.ip[12] == b'TTL=128':
            self.reg = self.winreg_processor_brand()
            if self.reg == "0" or self.reg == None:
                self.a1 = self.ip[1].decode('utf-8')
                self.a0 = 'MAQ' + self.ip[1][10:].decode('utf-8') + ' Indef'
                self.a2 = self.a3 = self.a4 = self.a5 = self.a6 = self.a7 = self.a8 = self.a9 = self.a10 = 'Sem Ref.'
                self.status = "Online"
                self.parametros()
            else:
                self.os = self.dados1()
                self.ver = self.dados2()
                a = re.match("(\d+)", self.ver[-2])
                b = round(int(a[0]))
                self.mem = (f'{b / 1024 ** 2:.0f}')
                self.user = self.cmd1()
                self.disc = self.cmd2()
                self.off = self.winreg_office()

                self.a0 = str(self.user[4])
                self.a1 = self.ip[1].decode('utf-8')
                if 'CEDASA' in self.user[-1]:
                    self.a2 = self.user[-1]
                    self.a9 = ' '.join(self.user[6:-1]) if self.user[5] == self.user[6] else ' '.join(self.user[5:-1])
                else:
                    self.a2 = 'CEDASA\Administrator'
                    self.a9 = ' '.join(self.user[6:]) if self.user[5] == self.user[6] else ' '.join(self.user[5:])
                self.a3 = self.os
                self.a4 = self.ver[-1]
                self.a5 = ' '.join(self.ver[3:-2])
                if self.off == "10":
                    self.a6 = 'Office XP'
                elif self.off == "11":
                    self.a6 = 'Office 2003'
                elif self.off == "12":
                    self.a6 = 'Office 2007'
                elif self.off == "14":
                    self.a6 = 'Office 2010'
                elif self.off == "15":
                    self.a6 = 'Office 2013'
                elif self.off == "16":
                    self.a6 = 'Office 365/2016'
                else:
                    self.a6 = 'Não reconhecido'
                self.a7 = self.mem + ' GB RAM'
                if self.disc == None:
                    self.a8 = 'Virtual Machine'
                else:
                    self.a8 = self.disc
                self.a10 = self.reg
                self.status = "Online"
                self.parametros()


    def parametros(self):
        self.tpl = (self.a0, self.a1, self.a2, self.a3, self.a4, self.a5, self.a6, self.a7, self.a8, self.a9, self.a10)
        print(self.tpl)
        self.insert()

    def insert(self):
        self.connect()
        self.sqli = """ insert or replace into inventario (hostname, ip, user, so, versao, arch, office, ram, disc, 
                        model, processor) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?); """
        self.cursor.execute(self.sqli, self.tpl)
        self.conn.commit(); print(f'inserido maq {self.maq}')
        self.disconnect()


App()

In [ ]:
import subprocess, winreg, re
from tkinter import *
from tkinter import ttk
import sqlite3
from openpyxl import Workbook

root = Tk()


class Funcs():
    def connect(self):
        self.conn = sqlite3.connect("../Scripts/inventário.bd")
        self.cursor = self.conn.cursor(); print('conectado')
    def disconnect(self):
        self.conn.close(); print('desconectado')
    def tabelas(self):
        self.connect()
        self.cursor.execute(""" Create table if not exists inventario ( 
            hostname CHAR(15) PRIMARY KEY,
            ip CHAR(20) NOT NULL,
            user CHAR(35) NOT NULL,
            so CHAR(50) NOT NULL,
            versao CHAR(15) NOT NULL,
            arch CHAR(15) NOT NULL,
            office CHAR(15) NOT NULL,
            ram CHAR(15) NOT NULL,
            disc CHAR(15) NOT NULL,
            model CHAR(35) NOT NULL,
            processor CHAR(50) NOT NULL
                  ); """)
        self.conn.commit(); print('Tab criada')
        self.disconnect()
    def select(self):
        try:
            self.connect()
            self.host = [(self.maq)]
            self.cursor.execute(""" select * from inventario where hostname = ? """, self.host)
            self.result = self.cursor.fetchall()
            self.disconnect()
        except:
            pass
        else:
            return self.result


class Prog():
    def winreg_processor_brand(self):
        try:
            target = r'\\{}'.format(self.maq)
            rem_reg = winreg.ConnectRegistry(target, winreg.HKEY_LOCAL_MACHINE)
            key = winreg.OpenKey(rem_reg, r"Hardware\Description\System\CentralProcessor\0")
            value = winreg.QueryValueEx(key, "ProcessorNameString")[0]
            winreg.CloseKey(key)
        except:
            pass
        else:
            self.processor_brand = value
            return self.processor_brand.strip()

    def winreg_office(self):
        try:
            target = r'\\{}'.format(self.maq)
            off_reg = winreg.ConnectRegistry(target, winreg.HKEY_LOCAL_MACHINE)
            key = winreg.OpenKey(off_reg, r"SOFTWARE\Classes\Word.Application\CurVer")
            value = winreg.QueryValueEx(key, None)[0]
            winreg.CloseKey(key)
        except:
            pass
        else:
            self.office = value[-2:]
            return self.office

    def dados1(self):
        try:
            pd1 = subprocess.run(f'wmic /node: {self.maq} OS get caption', stdout=subprocess.PIPE)
            rest1 = pd1.stdout.decode('UTF-8').split()
            self.stdout1 = ' '.join(rest1[1:])
        except:
            pass
        else:
            return self.stdout1

    def dados2(self):
        try:
            pd2 = subprocess.run(f'wmic /node: {self.maq} OS get Version, TotalVisibleMemorySize, OSArchitecture',
                                 stdout=subprocess.PIPE)
            rest2 = pd2.stdout.decode('UTF-8').split()
        except:
            pass
        else:
            self.stdoutd2 = rest2
            return self.stdoutd2

    def dados3(self):
        try:
            pd3 = subprocess.run(f'ping -n 1 {self.maq}', stdout=subprocess.PIPE)
            rest3 = pd3.stdout.split()
        except:
            return False
        else:
            self.stdoutd3 = rest3
            if self.stdoutd3[12] == b'TTL=128':
                return self.stdoutd3
            elif self.stdoutd3[12] == b'TTL=64':
                return self.stdoutd3
            elif self.stdoutd3[12] == b'TTL=255':
                return self.stdoutd3
            else:
                return False

    def cmd1(self):
        try:
            p1 = subprocess.run(f'wmic /node: {self.maq} computersystem get Caption, Manufacturer, Model, username',
                                stdout=subprocess.PIPE)
            res1 = p1.stdout.decode('UTF-8').split()
        except:
            pass
        else:
            self.stdout1 = res1
            return self.stdout1

    def cmd2(self):
        try:
            p2 = subprocess.run(f'wmic /node: {self.maq} logicaldisk get Name, size', stdout=subprocess.PIPE)
            res2 = p2.stdout.decode('UTF-8').split()
            a = re.match("(\d+)", res2[3])
            b = int(a[0])
        except:
            pass
        else:
            self.stdout2 = (f'{b / 1024**4:.2f} TB') if b > (1024**4) else (f'{b / 1024**3:.2f} GB')
            return self.stdout2


class App(Prog, Funcs):
    def __init__(self):
        self.root = root
        self.tabelas()
        self.tela()
        self.frames()
        self.labels()
        self.lista()
        root.mainloop()

    def tela(self):
        self.root.title('Inventário de Máquinas')
        self.root.configure(background='#1e3743')
        self.root.geometry("1900x1000")

    def frames(self):
        self.frame = Frame(self.root, bd=4, bg='#dfe3ee', highlightbackground='#759fe6', highlightthickness=3)
        self.frame.place(relx=0.0, rely=0.0, relwidth=1, relheight=1)

    def labels(self):
        self.listac = ttk.Treeview(self.frame, height=15, column=('col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7'
                                                                  , 'col8', 'col9', 'col10', 'col11'))
        self.h1 = ('Nome Máquina', 'IP', 'Usuário', 'SO', 'Versão', 'Arquitetura', 'Office', 'Memória', 'Disco',
                   'Modelo', 'Processador')

        self.listac.heading('#0', text='')
        self.listac.heading('#1', text=f'{self.h1[0]}')
        self.listac.heading('#2', text=f'{self.h1[1]}')
        self.listac.heading('#3', text=f'{self.h1[2]}')
        self.listac.heading('#4', text=f'{self.h1[3]}')
        self.listac.heading('#5', text=f'{self.h1[4]}')
        self.listac.heading('#6', text=f'{self.h1[5]}')
        self.listac.heading('#7', text=f'{self.h1[6]}')
        self.listac.heading('#8', text=f'{self.h1[7]}')
        self.listac.heading('#9', text=f'{self.h1[8]}')
        self.listac.heading('#10', text=f'{self.h1[9]}')
        self.listac.heading('#11', text=f'{self.h1[10]}')

        self.listac.column('#0', width=1)
        self.listac.column('#1', width=100)
        self.listac.column('#2', width=100)
        self.listac.column('#3', width=150)
        self.listac.column('#4', width=200)
        self.listac.column('#5', width=70)
        self.listac.column('#6', width=70)
        self.listac.column('#7', width=100)
        self.listac.column('#8', width=80)
        self.listac.column('#9', width=80)
        self.listac.column('#10', width=250)
        self.listac.column('#11', width=250)
        self.listac.place(relx=0.0, rely=0.0, relwidth=1, relheight=0.93)

        self.scroll = Scrollbar(self.frame, orient='vertical')
        self.listac.configure(yscroll=self.scroll.set)
        self.scroll.place(relx=0.99, rely=0.001, relwidth=0.009, relheight=0.928)

        self.entfax = Entry(self.frame)
        self.entfax.insert(0, '192.168.1')
        self.entfax.place(relx=0.115, rely=0.96, relwidth=0.082, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Definir Faixa', command=self.faixa)
        self.bt_frame.place(relx=0.20, rely=0.96, relwidth=0.06, relheight=0.025)

        self.entran1 = Entry(self.frame)
        self.entran1.insert(0, '1')
        self.entran1.place(relx=0.330, rely=0.96, relwidth=0.032, relheight=0.025)
        self.entran2 = Entry(self.frame)
        self.entran2.insert(0, '255')
        self.entran2.place(relx=0.365, rely=0.96, relwidth=0.032, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Definir Range', command=self.range)
        self.bt_frame.place(relx=0.40, rely=0.96, relwidth=0.06, relheight=0.025)

        self.entmaq = Entry(self.frame)
        self.entmaq.place(relx=0.515, rely=0.96, relwidth=0.082, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Inserir IP', command=self.insm)
        self.bt_frame.place(relx=0.60, rely=0.96, relwidth=0.06, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Apagar IP', command=self.delm)
        self.bt_frame.place(relx=0.663, rely=0.96, relwidth=0.06, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Atualizar tudo', command=self.atup)
        self.bt_frame.place(relx=0.8, rely=0.96, relwidth=0.06, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Extrair Excel', command=self.openxl)
        self.bt_frame.place(relx=0.863, rely=0.96, relwidth=0.06, relheight=0.025)

    def lista(self):
        self.listac.delete(*self.listac.get_children())
        self.connect()
        self.listasel = self.cursor.execute(""" select * from inventario order by ip """)
        for m in self.listasel:
            self.listac.insert('', END, values=m)
        self.disconnect()

    def listaex(self):
        self.connect()
        self.listasel = self.cursor.execute(""" select * from inventario order by ip """)
        for linha in self.listasel:
            self.plan1.append(linha)
        self.disconnect()

    def atup(self):
        self.maq = ''
        self.faixa()
        self.range()
        for x in range(self.ran1, self.ran2):
            self.maq = f'{self.fax}.{x}'
            print(self.maq)
            self.verificarSistema()
        self.lista()

    def insm(self):
        self.maq = ''
        self.maq = self.entmaq.get()
        self.verificarSistema()
        self.lista()

    def delm(self):
        self.maq = ''
        self.maq = (self.entmaq.get().split())
        print(self.maq)
        self.connect()
        self.sqli = """ delete from inventario where ip = ?; """
        self.cursor.execute(self.sqli, self.maq)
        self.conn.commit(); print(f'apagado maq {self.maq}')
        self.disconnect()
        self.lista()

    def faixa(self):
        self.fax = ''
        self.fax = self.entfax.get()

    def range(self):
        self.ran1 = 1
        self.ran1 = int(self.entran1.get())
        self.ran2 = 255
        self.ran2 = int(self.entran2.get())+1

    def openxl(self):
        self.arqexc = Workbook()
        self.plan1 = self.arqexc.active
        self.plan1.title = 'Inventário'
        self.plan1['A1'] = self.h1[0]
        self.plan1['B1'] = self.h1[1]
        self.plan1['C1'] = self.h1[2]
        self.plan1['D1'] = self.h1[3]
        self.plan1['E1'] = self.h1[4]
        self.plan1['F1'] = self.h1[5]
        self.plan1['G1'] = self.h1[6]
        self.plan1['H1'] = self.h1[7]
        self.plan1['I1'] = self.h1[8]
        self.plan1['J1'] = self.h1[9]
        self.plan1['K1'] = self.h1[10]

        self.listaex()
        self.arqexc.save(r"c:\temp\relatorio.xlsx")

    def verificarSistema(self):
        self.reg = '0'
        self.ip = self.dados3()
        if self.ip == False:
            self.status = 'Offline'

        elif self.ip[12] == b'TTL=64':
            self.a1 = self.ip[1].decode('utf-8')
            self.a0 = 'Linux ' + self.ip[1].decode('utf-8')
            self.a2 = self.a3 = self.a4 = self.a5 = self.a6 = self.a7 = self.a8 = self.a9 = self.a10= 'Sem Ref.'
            self.status = "Online"
            self.parametros()

        elif self.ip[12] == b'TTL=255':
            self.a1 = self.ip[1].decode('utf-8')
            self.a0 = 'Impressora ' + self.ip[1].decode('utf-8')
            self.a2 = self.a3 = self.a4 = self.a5 = self.a6 = self.a7 = self.a8 = self.a9 = self.a10 = 'Sem Ref.'
            self.status = "Online"
            self.parametros()

        elif self.ip[12] == b'TTL=128':
            self.reg = self.winreg_processor_brand()
            if self.reg == "0" or self.reg == None:
                self.a1 = self.ip[1].decode('utf-8')
                self.a0 = 'Indefinido' + self.ip[1].decode('utf-8')
                self.a2 = self.a3 = self.a4 = self.a5 = self.a6 = self.a7 = self.a8 = self.a9 = self.a10 = 'Sem Ref.'
                self.status = "Online"
                self.parametros()
            else:
                self.os = self.dados1()
                self.ver = self.dados2()
                a = re.match("(\d+)", self.ver[-2])
                b = round(int(a[0]))
                self.mem = (f'{b / 1024 ** 2:.0f}')
                self.user = self.cmd1()
                self.disc = self.cmd2()
                self.off = self.winreg_office()

                self.a0 = str(self.user[4])
                self.a1 = self.ip[1].decode('utf-8')
                if 'CEDASA' in self.user[-1]:
                    self.a2 = self.user[-1]
                    self.a9 = ' '.join(self.user[6:-1]) if self.user[5] == self.user[6] else ' '.join(self.user[5:-1])
                else:
                    self.a2 = 'CEDASA\Administrator'
                    self.a9 = ' '.join(self.user[6:]) if self.user[5] == self.user[6] else ' '.join(self.user[5:])
                self.a3 = self.os
                self.a4 = self.ver[-1]
                self.a5 = ' '.join(self.ver[3:-2])
                if self.off == "10":
                    self.a6 = 'Office XP'
                elif self.off == "11":
                    self.a6 = 'Office 2003'
                elif self.off == "12":
                    self.a6 = 'Office 2007'
                elif self.off == "14":
                    self.a6 = 'Office 2010'
                elif self.off == "15":
                    self.a6 = 'Office 2013'
                elif self.off == "16":
                    self.a6 = 'Office 365/2016'
                else:
                    self.a6 = 'Não reconhecido'
                self.a7 = self.mem + ' GB RAM'
                if self.disc == None:
                    self.a8 = 'Virtual Machine'
                else:
                    self.a8 = self.disc
                self.a10 = self.reg
                self.status = "Online"
                self.parametros()


    def parametros(self):
        self.tpl = (self.a0, self.a1, self.a2, self.a3, self.a4, self.a5, self.a6, self.a7, self.a8, self.a9, self.a10)
        print(self.tpl)
        self.insert()

    def insert(self):
        self.connect()
        self.sqli = """ insert or replace into inventario (hostname, ip, user, so, versao, arch, office, ram, disc, 
                        model, processor) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?); """
        self.cursor.execute(self.sqli, self.tpl)
        self.conn.commit(); print(f'inserido maq {self.maq}')
        self.disconnect()


App()

In [ ]:
import subprocess, winreg, re
from tkinter import *
from tkinter import ttk
import sqlite3

root = Tk()


class Funcs():
    def connect(self):
        self.conn = sqlite3.connect("../Scripts/inventário.bd")
        self.cursor = self.conn.cursor(); print('conectado')
    def disconnect(self):
        self.conn.close(); print('desconectado')
    def tabelas(self):
        self.connect()
        self.cursor.execute(""" Create table if not exists inventario ( 
            hostname CHAR(15) PRIMARY KEY,
            ip CHAR(20) NOT NULL,
            user CHAR(35) NOT NULL,
            so CHAR(50) NOT NULL,
            versao CHAR(15) NOT NULL,
            arch CHAR(15) NOT NULL,
            office CHAR(15) NOT NULL,
            ram CHAR(15) NOT NULL,
            disc CHAR(15) NOT NULL,
            model CHAR(35) NOT NULL,
            processor CHAR(50) NOT NULL
                  ); """)
        self.conn.commit(); print('Tab criada')
        self.disconnect()
    def select(self):
        try:
            self.connect()
            self.host = [(self.maq)]
            self.cursor.execute(""" select * from inventario where hostname = ? """, self.host)
            self.result = self.cursor.fetchall()
            self.disconnect()
        except:
            pass
        else:
            return self.result


class Prog():
    def winreg_processor_brand(self):
        try:
            target = r'\\{}'.format(self.maq)
            rem_reg = winreg.ConnectRegistry(target, winreg.HKEY_LOCAL_MACHINE)
            key = winreg.OpenKey(rem_reg, r"Hardware\Description\System\CentralProcessor\0")
            value = winreg.QueryValueEx(key, "ProcessorNameString")[0]
            winreg.CloseKey(key)
        except:
            pass
        else:
            self.processor_brand = value
            return self.processor_brand.strip()

    def winreg_office(self):
        try:
            target = r'\\{}'.format(self.maq)
            off_reg = winreg.ConnectRegistry(target, winreg.HKEY_LOCAL_MACHINE)
            key = winreg.OpenKey(off_reg, r"SOFTWARE\Classes\Word.Application\CurVer")
            value = winreg.QueryValueEx(key, None)[0]
            winreg.CloseKey(key)
        except:
            pass
        else:
            self.office = value[-2:]
            return self.office

    def dados1(self):
        try:
            pd1 = subprocess.run(f'wmic /node: {self.maq} OS get caption', stdout=subprocess.PIPE)
            rest1 = pd1.stdout.decode('UTF-8').split()
            self.stdout1 = ' '.join(rest1[1:])
        except:
            pass
        else:
            return self.stdout1

    def dados2(self):
        try:
            pd2 = subprocess.run(f'wmic /node: {self.maq} OS get Version, TotalVisibleMemorySize, OSArchitecture',
                                 stdout=subprocess.PIPE)
            rest2 = pd2.stdout.decode('UTF-8').split()
        except:
            pass
        else:
            self.stdoutd2 = rest2
            return self.stdoutd2

    def dados3(self):
        try:
            pd3 = subprocess.run(f'ping -n 1 {self.maq}', stdout=subprocess.PIPE)
            rest3 = pd3.stdout.split()
        except:
            return False
        else:
            self.stdoutd3 = rest3
            if self.stdoutd3[12] == b'TTL=128':
                return self.stdoutd3
            elif self.stdoutd3[12] == b'TTL=64':
                return self.stdoutd3
            elif self.stdoutd3[12] == b'TTL=255':
                return self.stdoutd3
            else:
                return False

    def cmd1(self):
        try:
            p1 = subprocess.run(f'wmic /node: {self.maq} computersystem get Caption, Manufacturer, Model, username',
                                stdout=subprocess.PIPE)
            res1 = p1.stdout.decode('UTF-8').split()
        except:
            pass
        else:
            self.stdout1 = res1
            return self.stdout1

    def cmd2(self):
        try:
            p2 = subprocess.run(f'wmic /node: {self.maq} logicaldisk get Name, size', stdout=subprocess.PIPE)
            res2 = p2.stdout.decode('UTF-8').split()
            a = re.match("(\d+)", res2[3])
            b = int(a[0])
        except:
            pass
        else:
            self.stdout2 = (f'{b / 1024**4:.2f} TB') if b > (1024**4) else (f'{b / 1024**3:.2f} GB')
            return self.stdout2


class App(Prog, Funcs):
    def __init__(self):
        self.root = root
        self.tabelas()
        self.tela()
        self.frames()
        self.labels()
        self.lista()
        root.mainloop()

    def tela(self):
        self.root.title('Inventário de Máquinas')
        self.root.configure(background='#1e3743')
        self.root.geometry("1900x1000")

    def frames(self):
        self.frame = Frame(self.root, bd=4, bg='#dfe3ee', highlightbackground='#759fe6', highlightthickness=3)
        self.frame.place(relx=0.01, rely=0.01, relwidth=0.98, relheight=0.98)

    def labels(self):
        self.listac = ttk.Treeview(self.frame, height=15, column=('col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7'
                                                                  , 'col8', 'col9', 'col10', 'col11'))
        self.listac.heading('#0', text='')
        self.listac.heading('#1', text='Nome Máquina')
        self.listac.heading('#2', text='IP')
        self.listac.heading('#3', text='Usuário')
        self.listac.heading('#4', text='SO')
        self.listac.heading('#5', text='Versão')
        self.listac.heading('#6', text='Arquitetura')
        self.listac.heading('#7', text='Office')
        self.listac.heading('#8', text='Memória')
        self.listac.heading('#9', text='Disco')
        self.listac.heading('#10', text='Modelo')
        self.listac.heading('#11', text='Processador')


        self.listac.column('#0', width=1)
        self.listac.column('#1', width=100)
        self.listac.column('#2', width=100)
        self.listac.column('#3', width=150)
        self.listac.column('#4', width=200)
        self.listac.column('#5', width=70)
        self.listac.column('#6', width=70)
        self.listac.column('#7', width=100)
        self.listac.column('#8', width=80)
        self.listac.column('#9', width=80)
        self.listac.column('#10', width=250)
        self.listac.column('#11', width=250)
        self.listac.place(relx=0.01, rely=0.01, relwidth=0.98, relheight=0.93)

        self.scroll = Scrollbar(self.frame, orient='vertical')
        self.listac.configure(yscroll=self.scroll.set)
        self.scroll.place(relx=0.98, rely=0.011, relwidth=0.009, relheight=0.928)

        self.entmaq = Entry(self.frame)
        self.entmaq.place(relx=0.615, rely=0.96, relwidth=0.082, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Inserir IP', command=self.insm)
        self.bt_frame.place(relx=0.70, rely=0.96, relwidth=0.06, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Apagar IP', command=self.delm)
        self.bt_frame.place(relx=0.763, rely=0.96, relwidth=0.06, relheight=0.025)

        self.bt_frame = Button(self.frame, text='Atualizar tudo', command=self.atup)
        self.bt_frame.place(relx=0.9, rely=0.96, relwidth=0.06, relheight=0.025)

    def lista(self):
        self.listac.delete(*self.listac.get_children())
        self.connect()
        lista = self.cursor.execute(""" select * from inventario order by ip """)
        for m in lista:
            self.listac.insert('', END, values=m)
        self.disconnect()

    def atup(self):
        self.maq = ''
        for x in range(239,255):
            self.maq = f'192.168.1.{x}'
            print(self.maq)
            self.verificarSistema()
        self.lista()

    def insm(self):
        self.maq = ''
        self.maq = self.entmaq.get()
        self.verificarSistema()
        self.lista()

    def delm(self):
        self.maq = ''
        self.maq = (self.entmaq.get().split())
        print(self.maq)
        self.connect()
        self.sqli = """ delete from inventario where ip = ?; """
        self.cursor.execute(self.sqli, self.maq)
        self.conn.commit(); print(f'apagado maq {self.maq}')
        self.disconnect()
        self.lista()

    def verificarSistema(self):
        self.reg = '0'
        self.ip = self.dados3()
        if self.ip == False:
            self.status = 'Offline'

        elif self.ip[12] == b'TTL=64':
            self.a1 = self.ip[1].decode('utf-8')
            self.a0 = 'Linux ' + self.ip[1].decode('utf-8')
            self.a2 = self.a3 = self.a4 = self.a5 = self.a6 = self.a7 = self.a8 = self.a9 = self.a10= 'Sem Ref.'
            self.status = "Online"
            self.parametros()

        elif self.ip[12] == b'TTL=255':
            self.a1 = self.ip[1].decode('utf-8')
            self.a0 = 'Impressora ' + self.ip[1].decode('utf-8')
            self.a2 = self.a3 = self.a4 = self.a5 = self.a6 = self.a7 = self.a8 = self.a9 = self.a10 = 'Sem Ref.'
            self.status = "Online"
            self.parametros()

        elif self.ip[12] == b'TTL=128':
            self.reg = self.winreg_processor_brand()
            if self.reg == "0" or self.reg == None:
                self.a1 = self.ip[1].decode('utf-8')
                self.a0 = 'Indefinido' + self.ip[1].decode('utf-8')
                self.a2 = self.a3 = self.a4 = self.a5 = self.a6 = self.a7 = self.a8 = self.a9 = self.a10 = 'Sem Ref.'
                self.status = "Online"
                self.parametros()
            else:
                self.os = self.dados1()
                self.ver = self.dados2()
                a = re.match("(\d+)", self.ver[-2])
                b = round(int(a[0]))
                self.mem = (f'{b / 1024 ** 2:.0f}')
                self.user = self.cmd1()
                self.disc = self.cmd2()
                self.off = self.winreg_office()

                self.a0 = str(self.user[4])
                self.a1 = self.ip[1].decode('utf-8')
                if 'CEDASA' in self.user[-1]:
                    self.a2 = self.user[-1]
                    self.a9 = ' '.join(self.user[6:-1]) if self.user[5] == self.user[6] else ' '.join(self.user[5:-1])
                else:
                    self.a2 = 'CEDASA\Administrator'
                    self.a9 = ' '.join(self.user[6:]) if self.user[5] == self.user[6] else ' '.join(self.user[5:])
                self.a3 = self.os
                self.a4 = self.ver[-1]
                self.a5 = ' '.join(self.ver[3:-2])
                if self.off == "10":
                    self.a6 = 'Office XP'
                elif self.off == "11":
                    self.a6 = 'Office 2003'
                elif self.off == "12":
                    self.a6 = 'Office 2007'
                elif self.off == "14":
                    self.a6 = 'Office 2010'
                elif self.off == "15":
                    self.a6 = 'Office 2013'
                elif self.off == "16":
                    self.a6 = 'Office 365/2016'
                else:
                    self.a6 = 'Não reconhecido'
                self.a7 = self.mem + ' GB RAM'
                if self.disc == None:
                    self.a8 = 'Virtual Machine'
                else:
                    self.a8 = self.disc
                self.a10 = self.reg
                self.status = "Online"
                self.parametros()


    def parametros(self):
        self.tpl = (self.a0, self.a1, self.a2, self.a3, self.a4, self.a5, self.a6, self.a7, self.a8, self.a9, self.a10)
        print(self.tpl)
        self.insert()

    def insert(self):
        self.connect()
        self.sqli = """ insert or replace into inventario (hostname, ip, user, so, versao, arch, office, ram, disc, 
                        model, processor) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?); """
        self.cursor.execute(self.sqli, self.tpl)
        self.conn.commit(); print(f'inserido maq {self.maq}')
        self.disconnect()


App()

In [ ]:
import subprocess, winreg
from tkinter import *

root = Tk()


class prog():
    def winreg_processor_brand(self):
        target = r'\\{}'.format(self.maq)
        rem_reg = winreg.ConnectRegistry(target, winreg.HKEY_LOCAL_MACHINE)
        key = winreg.OpenKey(rem_reg, r"Hardware\Description\System\CentralProcessor\0")
        value = winreg.QueryValueEx(key, "ProcessorNameString")[0]
        winreg.CloseKey(key)
        self.processor_brand = value
        return self.processor_brand.strip()

    def dados(self):
        pd = subprocess.run(f'wmic /node: {self.maq} OS get OSArchitecture', stdout=subprocess.PIPE)
        rest = pd.stdout.decode('UTF-8').split()
        self.stdout = rest
        return self.stdout

    def dados1(self):
        pd1 = subprocess.run(f'wmic /node: {self.maq} OS get caption', stdout=subprocess.PIPE)
        rest1 = pd1.stdout.decode('UTF-8').split()
        self.stdoutd1 = rest1
        return self.stdoutd1

    def dados2(self):
        pd2 = subprocess.run(f'wmic /node: {self.maq} OS get Version', stdout=subprocess.PIPE)
        rest2 = pd2.stdout.decode('UTF-8').split()
        self.stdoutd2 = rest2
        return self.stdoutd2

    def dados3(self):
        pd3 = subprocess.run(f'ping {self.maq}', stdout=subprocess.PIPE)
        rest3 = pd3.stdout.split()
        self.stdoutd3 = rest3
        return self.stdoutd3

    def cmd(self):
        import re
        p0 = subprocess.run(f'wmic /node: {self.maq} OS get TotalVisibleMemorySize', stdout=subprocess.PIPE)
        res0 = p0.stdout.decode('UTF-8').split()  # captura stdout
        a = re.match("(\d+)", res0[1])
        b = round(int(a[0]))
        self.stdout = (f'{b / 1024 ** 2:.0f}')
        return self.stdout

    def cmd1(self):
        p1 = subprocess.run(f'wmic /node: {self.maq} computersystem get username, Model, Caption', stdout=subprocess.PIPE)
        res1 = p1.stdout.decode('UTF-8').split()
        self.stdout1 = res1
        return self.stdout1

    def cmd2(self):
        import re
        p2 = subprocess.run(f'wmic /node: {self.maq} logicaldisk get Name, size', stdout=subprocess.PIPE)
        res2 = p2.stdout.decode('UTF-8').split()
        a = re.match("(\d+)", res2[3])
        b = int(a[0])
        self.stdout2 = (f'{b / 1024**3:.2f}')
        return self.stdout2


class App(prog):
    def __init__(self):
        self.root = root
        self.tela()
        self.frames()
        self.verificarSistema()
        self.labels()
        root.mainloop()

    def tela(self):
        self.root.title('Inventário')
        self.root.configure(background='#1e3743')
        self.root.geometry("800x600")
        self.root.resizable(False, False)

    def frames(self):
        self.frame = Frame(self.root, bd=4, bg='#dfe3ee', highlightbackground='#759fe6', highlightthickness=3)
        self.frame.place(relx=0.01, rely=0.01, relwidth=0.98, relheight=0.98)

    def labels(self):
        rx = ry = 0.03
        for k, v in self.info.items():
            self.lb_frame = Label(self.frame, text=f'{k}', bg='#dfe3ee')
            self.lb_frame.place(relx=rx, rely=ry)

            self.lb_frame = Label(self.frame, text=f'{v}', bg='#dfe3ee')
            self.lb_frame.place(relx=rx+0.25, rely=ry)
            ry += 0.05

    def verificarSistema(self):
        self.maq = 'cedasaweb'  # socket.gethostname()
        self.reg = self.winreg_processor_brand()
        self.ram = self.dados()
        self.os = self.dados1()
        self.ver = self.dados2()
        self.ip = self.dados3()
        self.mem = self.cmd()
        self.user = self.cmd1()
        self.disc = self.cmd2()

        self.info = {}
        self.info['hostname'] = self.user[3]
        self.info['IP'] = self.ip[2].decode('utf-8')
        self.info['username'] = self.user[-1]
        self.info['OS'] = ' '.join(self.os[1:])
        self.info['version'] = self.ver[1]
        self.info['arch'] = ' '.join(self.ram[1:])
        self.info['Memória'] = self.mem + ' GB RAM'
        self.info['Disco'] = self.disc +' GB'
        self.info['Modelo'] = ' '.join(self.user[4:-1])
        self.info['processor'] = self.reg
        # print(self.ip[13])


App()


In [ ]:
import platform, socket, psutil
import os, sys
import subprocess
import winreg
from tkinter import *

root = Tk()


class prog():


    def winreg_processor_brand(self):
        target = r'\\maq248'
        rem_reg = winreg.ConnectRegistry(target, winreg.HKEY_LOCAL_MACHINE)
        key = winreg.OpenKey(rem_reg, r"Hardware\Description\System\CentralProcessor\0")
        value = winreg.QueryValueEx(key, "ProcessorNameString")[0]
        winreg.CloseKey(key)
        self.processor_brand = value
        return self.processor_brand.strip()

    def dados(self):
        OS = platform.system() + ' ' + platform.release() + ' ' + platform.win32_edition()
        print(OS)
        # p1 = subprocess.run(f'wmic /node: {self.maq} computersystem get username, Model, Caption',
        #                     stdout=subprocess.PIPE)
        # res1 = p1.stdout.decode('UTF-8').split()
        # self.stdout1 = res1
        # return self.stdout1

        #response = os.system("ping -c 1 " + self.maq)
        response = os.system(r'ping -n 1 {} > C:\Users\vinicius.bertanha\PycharmProjects\inventario\ping.txt'.format(self.maq))

        # and then check the response...
        if response == 0:
            print(self.maq, 'is up!')
        else:
            print(self.maq, 'is down!')
        print(response)

    def cmd(self):
        # Abre um subprocesso ping usando a sintaxe Windows
        p0 = subprocess.run(f'wmic /node: {self.maq} path softwarelicensingservice get OA3xOriginalProductKey', stdout=subprocess.PIPE)
        res0 = p0.stdout.decode('UTF-8').split()  # captura stdout
        self.stdout = res0[1]
        return self.stdout

    def cmd1(self):
        # Abre um subprocesso ping usando a sintaxe Windows
        p1 = subprocess.run(f'wmic /node: {self.maq} computersystem get username, Model, Caption', stdout=subprocess.PIPE)
        res1 = p1.stdout.decode('UTF-8').split()
        self.stdout1 = res1
        return self.stdout1

    def cmd2(self):
        import re
        p2 = subprocess.run(f'wmic /node: {self.maq} logicaldisk get Name, size', stdout=subprocess.PIPE)
        res2 = p2.stdout.decode('UTF-8').split()
        a = re.match("(\d+)", res2[3])
        b = int(a[0])
        self.stdout2 = (f'{b / 1024**3:.2f}')
        return self.stdout2

    def get_ip(self):
        self.maq = 'maq248'  # socket.gethostname()
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        try:
            s.connect(('10.255.255.255', 1))
            self.IP = s.getsockname()[0]
        except Exception:
            self.IP = '127.0.0.1'
        finally:
            s.close()
        return self.IP


class App(prog):
    def __init__(self):
        self.root = root
        self.tela()
        self.frames()
        self.verificarSistema()
        self.labels()
        self.dados()
        root.mainloop()

    def tela(self):
        self.root.title('Inventário')
        self.root.configure(background='#1e3743')
        self.root.geometry("800x600")
        self.root.resizable(False, False)

    def frames(self):
        self.frame = Frame(self.root, bd=4, bg='#dfe3ee', highlightbackground='#759fe6', highlightthickness=3)
        self.frame.place(relx=0.01, rely=0.01, relwidth=0.98, relheight=0.98)

    def labels(self):
        print(self.info)
        rx = ry = 0.03
        for k, v in self.info.items():
            self.lb_frame = Label(self.frame, text=f'{k}', bg='#dfe3ee')
            self.lb_frame.place(relx=rx, rely=ry)

            self.lb_frame = Label(self.frame, text=f'{v}', bg='#dfe3ee')
            self.lb_frame.place(relx=rx+0.25, rely=ry)
            ry += 0.05

    def verificarSistema(self):
        self.reg = self.winreg_processor_brand()
        self.IP = self.get_ip()
        self.chw = self.cmd()
        self.user = self.cmd1()
        self.disc = self.cmd2()

        self.info = {}
        self.info['hostname'] = self.user[3]
        self.info['IP'] = self.IP
        self.info['username'] = self.user[-1]
        self.info['OS'] = platform.system() + ' ' + platform.release() + ' ' + platform.win32_edition()
        self.info['version'] = platform.version()
        self.info['ChaveWin'] = self.chw
        self.info['processor'] = self.reg
        self.info['arch'] = platform.machine()
        self.info['Memória'] = str(round(psutil.virtual_memory().total / (1024.0 ** 3))) + 'MB RAM'
        self.info['Disco'] = self.disc
        self.info['test'] = ' '.join(self.user[4:-1])


App()


In [ ]:
import platform, socket, psutil, re
# import os, sys
import subprocess
import winreg
from tkinter import *

root = Tk()

class prog():
    def winreg_processor_brand(self):
        key = winreg.OpenKey(winreg.HKEY_LOCAL_MACHINE, r"Hardware\Description\System\CentralProcessor\0")
        value = winreg.QueryValueEx(key, "ProcessorNameString")[0]
        winreg.CloseKey(key)
        self.processor_brand = value
        return self.processor_brand.strip()


    def cmd(self):
        # Abre um subprocesso ping usando a sintaxe Windows
        p1 = subprocess.run('wmic path softwarelicensingservice get OA3xOriginalProductKey', stdout=subprocess.PIPE)
        res1 = p1.stdout.decode('UTF-8').split()  # captura stdout
        self.stdout = res1[1]
        return self.stdout

    def cmd2(self):
        p2 = subprocess.run('wmic logicaldisk get Name, size', stdout=subprocess.PIPE)
        res2 = p2.stdout.decode('UTF-8').split()
        a = re.match("(\d+)", res2[3])
        b = int(a[0])
        self.stdout1 = (f'{b / 1024**3:.2f}')
        return self.stdout1


    def get_ip(self):
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        try:
            s.connect(('10.255.255.255', 1))
            self.IP = s.getsockname()[0]
        except Exception:
            self.IP = '127.0.0.1'
        finally:
            s.close()
        return self.IP


    def verificarSistema(self):
        self.reg = self.winreg_processor_brand()
        self.IP = self.get_ip()
        self.chw = self.cmd()
        self.disc = self.cmd2()

        self.info = {}
        self.info['OS'] = platform.system()
        self.info['release'] = platform.release()
        self.info['edition'] = platform.win32_edition()
        self.info['version'] = platform.version()
        self.info['processor'] = self.reg
        self.info['arch'] = platform.machine()
        self.info['ram'] = str(round(psutil.virtual_memory().total / (1024.0 ** 3)))
        self.info['hostname'] = socket.gethostname()
        self.info['IP'] = self.IP
        self.info['arch2'] = platform.architecture()
        self.info['ChaveWin'] = self.chw
        self.info['Disco'] = self.disc


class App(prog):
    def __init__(self):
        self.root = root
        self.tela()
        self.frames_da_tela()
        self.verificarSistema()
        self.labels()
        root.mainloop()

    def tela(self):
        self.root.title('inventário')
        self.root.configure(background='#1e3743')
        self.root.geometry("800x600")
        self.root.resizable(False, False)

    def frames_da_tela(self):
        self.frame = Frame(self.root, bd=4, bg='#dfe3ee', highlightbackground='#759fe6', highlightthickness=3)
        self.frame.place(relx=0.01, rely=0.01, relwidth=0.98, relheight=0.98)

    def labels(self):
        # print(self.info)
        rx=ry=0.03
        for k, v in self.info.items():
            self.lb_frame = Label(self.frame, text=f'{k}', bg='#dfe3ee')
            self.lb_frame.place(relx=rx, rely=ry)

            self.lb_frame = Label(self.frame, text=f'{v}', bg='#dfe3ee')
            self.lb_frame.place(relx=rx+0.25, rely=ry)
            ry+=0.05

App()


In [ ]:
import platform, socket, psutil
import os, sys
import subprocess
import winreg
from tkinter import *


def _read_windows_registry_key(key_name, field_name):
    key = winreg.OpenKey(winreg.HKEY_LOCAL_MACHINE, key_name)
    value = winreg.QueryValueEx(key, field_name)[0]
    winreg.CloseKey(key)
    return value

def winreg_processor_brand():
    processor_brand = _read_windows_registry_key(r"Hardware\Description\System\CentralProcessor\0",
                                                 "ProcessorNameString")
    return processor_brand.strip()

reg = winreg_processor_brand()

def cmd():
    # Abre um subprocesso ping usando a sintaxe Windows
    p1 = subprocess.run('wmic path softwarelicensingservice get OA3xOriginalProductKey', stdout=subprocess.PIPE)
    res1 = p1.stdout.decode('UTF-8').split()  # captura stdout
    stdout = res1
    return stdout

def cmd2():
    p2 = subprocess.run('wmic logicaldisk get Name, size', stdout=subprocess.PIPE)
    res2 = p2.stdout.decode('UTF-8').split()
    stdout = res2
    return stdout


def get_ip():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:
        s.connect(('10.255.255.255', 1))
        IP = s.getsockname()[0]
    except Exception:
        IP = '127.0.0.1'
    finally:
        s.close()
    return IP

IP = get_ip()
chw = cmd()
disc = cmd2()

def verificarSistema():
    info = {}
    info['OS'] = platform.system()
    info['release'] = platform.release()
    info['edition'] = platform.win32_edition()
    info['version'] = platform.version()
    info['processor'] = reg
    info['arch'] = platform.machine()
    info['ram'] = str(round(psutil.virtual_memory().total / (1024.0 ** 3)))
    info['hostname'] = socket.gethostname()
    info['IP'] = IP
    info['arch2'] = platform.architecture()
    info['ChaveWin'] = chw
    info['Disco'] = disc

    print(info)


verificarSistema()


In [ ]:
import platform, socket, psutil
import os, sys
import subprocess
import winreg
from tkinter import *


def _read_windows_registry_key(key_name, field_name):
    key = winreg.OpenKey(winreg.HKEY_LOCAL_MACHINE, key_name)
    value = winreg.QueryValueEx(key, field_name)[0]
    winreg.CloseKey(key)
    return value

def winreg_processor_brand():
    processor_brand = _read_windows_registry_key(r"Hardware\Description\System\CentralProcessor\0",
                                                 "ProcessorNameString")
    return processor_brand.strip()

reg = winreg_processor_brand()

def cmd():

    # Abre um subprocesso ping usando a sintaxe Linux
    # p = subprocess.run(['ping', '-c', '4', site], stdout=subprocess.PIPE)
    # Abre um subprocesso ping usando a sintaxe Windows
    p1 = subprocess.run('wmic path softwarelicensingservice get OA3xOriginalProductKey, OA3xOriginalProductKeyDescription', stdout=subprocess.PIPE)
    p2 = subprocess.run('wmic logicaldisk get description, Name, size', stdout=subprocess.PIPE)

    stdout = [(p1.stdout.split())] #, (p2.stdout.split())]  # captura stdout
    print(stdout[0][0])

    # Retorna o texto do terminal se hover erro retorna vazio
    return stdout.decode('UTF-8') if p1.returncode == 0 else ''
    # No windows
    # return stdout.decode('ISO-8859-1') if p.returncode == 0 else ''    return stdout


def get_ip():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:
        # doesn't even have to be reachable
        s.connect(('10.255.255.255', 1))
        IP = s.getsockname()[0]
    except Exception:
        IP = '127.0.0.1'
    finally:
        s.close()
    return IP

IP = get_ip()

test = cmd()
print(test)

def verificarSistema():
    info = {}
    info['OS'] = platform.system()
    info['release'] = platform.release()
    info['edition'] = platform.win32_edition()
    info['version'] = platform.version()
    info['processor'] = reg
    info['arch'] = platform.machine()
    info['ram'] = str(round(psutil.virtual_memory().total / (1024.0 ** 3)))
    info['hostname'] = socket.gethostname()
    info['IP'] = IP
    info['arch2'] = platform.architecture()
    info['ID'] = test

    print(info)


verificarSistema()



In [ ]:
import platform, socket, psutil
import os, sys
import subprocess
import winreg
from tkinter import *



def _read_windows_registry_key(key_name, field_name):
    key = winreg.OpenKey(winreg.HKEY_LOCAL_MACHINE, key_name)
    value = winreg.QueryValueEx(key, field_name)[0]
    winreg.CloseKey(key)
    return value

def winreg_processor_brand():
    processor_brand = _read_windows_registry_key(r"Hardware\Description\System\CentralProcessor\0",
                                                 "ProcessorNameString")
    return processor_brand.strip()

reg = winreg_processor_brand()


def get_ip():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:
        # doesn't even have to be reachable
        s.connect(('10.255.255.255', 1))
        IP = s.getsockname()[0]
    except Exception:
        IP = '127.0.0.1'
    finally:
        s.close()
    return IP

IP = get_ip()

def winID():
    current_machine_id = subprocess.check_output('wmic csproduct get uuid').decode().split('\n')[1].strip()
    return current_machine_id

ID = winID()

def verificarSistema():
    info = {}
    info['OS'] = platform.system()
    info['release'] = platform.release()
    info['edition'] = platform.win32_edition()
    info['version'] = platform.version()
    info['processor'] = reg
    info['arch'] = platform.machine()
    info['ram'] = str(round(psutil.virtual_memory().total / (1024.0 ** 3)))
    info['hostname'] = socket.gethostname()
    info['IP'] = IP
    info['arch2'] = platform.architecture()
    info['ID'] = ID
    info['ID2'] = wmic path softwarelicensingservice get OA3xOriginalProductKey

    print(info)


verificarSistema()
